# MinHashing Example

Using the IMDB data, create different feature matrices and demonstrate performance improvement when using minhash signatures as features rather than the full set of actors.

In [1]:
%matplotlib inline

In [2]:
import json
import random
import pandas as pd

from scipy.sparse import lil_matrix

from sklearn.metrics import jaccard_score

In [3]:
# For testing, set abridge to true and a small dataset limit
#. Can run with MinHash without abridging your data
abridge = True
dataset_limit = 10000

In [4]:
actor_id_to_name_map = {}     # Map Actor IDs to actor names
actor_id_to_index_map = {}    # Map actor IDs to a unique index of known actors
index_to_actor_ids = []       # Array mapping unique index back to actor ID (invert of actor_id_to_index_map)

index_counter = 0    # Unique actor index; increment for each new actor
known_actors = set()

movie_actor_list = [] # List of all our movies and their actors

test_count = 0
with open("../data/imdb_recent_movies.json", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
            
        # Keep track of all the actors in this movie
        for actor_id,actor_name in zip(this_movie['actor_ids'],this_movie['actor_names']):
            
            # Keep names and IDs
            actor_id_to_name_map[actor_id] = actor_name
            
            # If we've seen this actor before, skip...
            if actor_id in known_actors:
                continue
                
            # ... Otherwise, add to known actor set and create new index for them
            known_actors.add(actor_id)
            actor_id_to_index_map[actor_id] = index_counter
            index_to_actor_ids.append(actor_id)
            index_counter += 1
            
        # Finished with this film
        movie_actor_list.append({
            "movie": this_movie["title_name"],
            "actors": set(this_movie['actor_ids'])
        })
        
        # If abrdiged, test for limit
        test_count += 1        
        if abridge and test_count > dataset_limit:
            break

In [5]:
print("Known Actors:", len(known_actors))

Known Actors: 17307


## Failure with Regular Matrix

If we have too much data, trying to keep track of the feature vectors gets hard.

Here, we do it naively, creating vectors of False for all movies and flipping values to True for actors in each movie.

__NOTE__ THIS WILL FAIL FOR LARGE DATA

In [6]:
# With regular matrix
matrix_rows = []

for movie in movie_actor_list:
    # Generate a row with only zeros
    this_actor_vector = [False] * len(known_actors)
    
    for actor_id in movie["actors"]:
        this_index = actor_id_to_index_map[actor_id]
        this_actor_vector[this_index] = True
        
    matrix_rows.append(this_actor_vector)

In [7]:
df = pd.DataFrame(
    matrix_rows, 
    index=[m["movie"] for m in movie_actor_list],
    columns=[index_to_actor_ids[i] for i in range(len(known_actors))]
)
df

,nm0001379,nm0000953,nm2505816,nm0706670,nm1209916,nm1283523,nm0542403,nm0831685,nm0348435,nm0523722,...,nm0922563,nm1181191,nm0506165,nm1678785,nm6934209,nm0156470,nm0702666,nm1220448,nm0028067,nm0144914
The Other Side of the Wind,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
November 1828,False,False,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
The Drive to Win,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
The Naked Monster,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
El día de los albañiles 2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wang choi jau sau,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False
"Wood & Stock: Sexo, Orégano e Rock'n'Roll",False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
¡Alerta!... La justicia de Rojo,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Streetwise,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Generate Same DataFrame using Sparse Matrics

The above will break if you have too much data. We can get around that partially with sparse matrices, where we only store the non-zero elements of the feature matrix and their indices.

In [8]:
# With sparse matrix, initialize to size of Movies x Actors of 0s
matrix_sparse = lil_matrix((len(movie_actor_list), len(known_actors)), dtype=bool)

# Update the matrix, movie by movie, setting non-zero values for the appropriate actors
for row,movie in enumerate(movie_actor_list):   
    for actor_id in movie["actors"]:
        this_index = actor_id_to_index_map[actor_id]
        matrix_sparse[row,this_index] = 1

In [9]:
df = pd.DataFrame.sparse.from_spmatrix(
    matrix_sparse, 
    index=[m["movie"] for m in movie_actor_list],
    columns=[index_to_actor_ids[i] for i in range(len(known_actors))]
)
df

,nm0001379,nm0000953,nm2505816,nm0706670,nm1209916,nm1283523,nm0542403,nm0831685,nm0348435,nm0523722,...,nm0922563,nm1181191,nm0506165,nm1678785,nm6934209,nm0156470,nm0702666,nm1220448,nm0028067,nm0144914
The Other Side of the Wind,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
November 1828,0,0,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
The Drive to Win,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
The Naked Monster,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
El día de los albañiles 2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wang choi jau sau,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
"Wood & Stock: Sexo, Orégano e Rock'n'Roll",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
¡Alerta!... La justicia de Rojo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Streetwise,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Test Jaccard Scores

Use the columns of our data frame to evaluate Jaccard similarity

In [10]:
# Expect to be 1, as Jaccard similarity with yourself should be 1.0
jaccard_score(df.loc["The Drive to Win"], df.loc["The Drive to Win"])

1.0

In [11]:
# Test with two similar movies
jaccard_score(df.loc["Moulin Rouge!"], df.loc["Big Fish"])

0.2

In [12]:
# Find the most similar movies to this query movie using the full Jaccard set
query_row = df.loc["Star Wars: Episode II - Attack of the Clones"]
similarities = []

# Check every row in the DF
for idx,row in df.iterrows():
    score = jaccard_score(query_row, row)
    similarities.append((idx, score))


In [13]:
most_sim = sorted(similarities, key=lambda d: d[1], reverse=True)[:10]
most_sim

[('Star Wars: Episode II - Attack of the Clones', 1.0),
 ('Star Wars: Episode III - Revenge of the Sith', 0.5),
 ('Awake', 0.25),
 ('Life as a House', 0.25),
 ('Young Adam', 0.25),
 ('Down with Love', 0.25),
 ('Nora', 0.2),
 ('Moulin Rouge!', 0.2),
 ('Quantum Quest: A Cassini Space Odyssey', 0.2),
 ('Big Fish', 0.2)]

## Convert to MinHash Reduced Dimension

For min-hashing, we first generate some number of permutations that will drive our "min hash" of the first non-zero element in the permuted column set (i.e., after shuffling columns, find the first permuted column idex with a non-zero element).

Note that, here, we _never_ store the full matrix of Movies X Actors.

In [14]:
permutations = []

# How many hashing functions to use
#  What happens as you change this number?
#  Larger values make things slower but provide better Jaccard estimates
num_minhashes = 10

# Generate permutations for hashing d-dimensional features into k minhash dimensions
for i in range(num_minhashes):
    column_indices = list(range(len(known_actors)))
    
    # Shuffle the original indices
    random.shuffle(column_indices)
    
    # Generate a map of the actual index to its permuted index
    permutations.append({j:i for i,j in enumerate(column_indices)})

In [16]:
minhash_rows = []

test_count = 0
with open("../data/imdb_recent_movies.json", "r") as in_file:
    for line in in_file:
        
        this_movie = json.loads(line)
            
        this_movie_actor_indices = []
        for actor_id in this_movie['actor_ids']:
            this_movie_actor_indices.append(actor_id_to_index_map[actor_id])
            
        minhash_sig = []
        for permutation in permutations:
            
            # Find the first index of actors in this movie using the permuted indices map
            permed_index = sorted([permutation[i] for i in this_movie_actor_indices])[0]
            minhash_sig.append(permed_index)

            # Could do this instead, but it's slower in Python
#             for permed_index,rand_index in enumerate(permutation):
#                 if rand_index in this_movie_actor_indices:
#                     minhash_sig.append(permed_index)
#                     break
        
        minhash_rows.append(minhash_sig)
        
        
        test_count += 1        
        if abridge and test_count > dataset_limit:
            break

In [17]:
minhash_df = pd.DataFrame(
    minhash_rows,
    index=[m["movie"] for m in movie_actor_list],
)
minhash_df

,0,1,2,3,4,5,6,7,8,9
The Other Side of the Wind,10586,6201,3288,539,11046,2066,7085,13096,5254,13734
November 1828,213,5715,4756,3234,2231,469,1456,2234,175,4376
The Drive to Win,7203,7931,474,11750,3692,8915,5910,3454,7081,6572
The Naked Monster,7013,1519,5609,3973,148,3763,3726,6394,2187,920
El día de los albañiles 2,2687,7592,1840,3937,3066,1342,1337,286,4396,11845
...,...,...,...,...,...,...,...,...,...,...
Wang choi jau sau,302,424,1550,2585,145,3582,2844,512,6427,2348
"Wood & Stock: Sexo, Orégano e Rock'n'Roll",1396,8831,1787,3662,9719,646,2866,13104,2641,3109
¡Alerta!... La justicia de Rojo,5890,3026,5702,1370,1224,3243,424,198,1794,1075
Streetwise,1213,6997,4522,4885,3854,6421,6852,10221,13582,4629


In [18]:
query_row = minhash_df.loc["Star Wars: Episode II - Attack of the Clones"]
minh_similarities = []

for idx,row in minhash_df.iterrows():
    score = sum(query_row == row) / len(row)
    minh_similarities.append((idx, score))
    

In [19]:
most_sim_mh = sorted(minh_similarities, key=lambda d: d[1], reverse=True)[:20]
most_sim_mh

[('Star Wars: Episode II - Attack of the Clones', 1.0),
 ('Star Wars: Episode III - Revenge of the Sith', 0.3),
 ('Nora', 0.2),
 ('Awake', 0.2),
 ('Black Hawk Down', 0.2),
 ('Young Adam', 0.2),
 ('Moulin Rouge!', 0.1),
 ('Life as a House', 0.1),
 ('Down with Love', 0.1),
 ('Quantum Quest: A Cassini Space Odyssey', 0.1),
 ('Big Fish', 0.1),
 ('Shattered Glass', 0.1),
 ('Crimson Rivers 2: Angels of the Apocalypse', 0.1),
 ('The Other Side of the Wind', 0.0),
 ('November 1828', 0.0),
 ('The Drive to Win', 0.0),
 ('The Naked Monster', 0.0),
 ('El día de los albañiles 2', 0.0),
 ('En tres y dos', 0.0),
 ('Grizzly II: Revenge', 0.0)]